In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
cabbage_df = pd.read_csv("data-files/merged_df_1.csv")

In [3]:
cabbage_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20591 entries, 0 to 20590
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   DATE            20591 non-null  object 
 1   거래단위            20591 non-null  object 
 2   도매시장            20591 non-null  object 
 3   산지-광역시도         20591 non-null  object 
 4   등급              20591 non-null  object 
 5   단위_무게(kg)       20591 non-null  int64  
 6   1kg_물량          20591 non-null  int64  
 7   1kg_평균가격        20591 non-null  float64
 8   weighted_price  20591 non-null  float64
 9   광역시도            20591 non-null  object 
 10  평균기온            20591 non-null  float64
 11  평균풍속            20591 non-null  float64
 12  평균강수량           20591 non-null  float64
dtypes: float64(5), int64(2), object(6)
memory usage: 2.0+ MB


# 데이터 부족한 도매시장제거

In [4]:
cabbage_df['DATE'] = pd.to_datetime(cabbage_df['DATE'])

# Month 열을 추가하여 날짜에서 월 정보 확인
cabbage_df['Month'] = cabbage_df['DATE'].dt.month

# 각 도매시장별 월별 데이터 개수를 확인
monthly_data_availability = cabbage_df.groupby(['도매시장', 'Month']).size().unstack(fill_value=0)

# 모든 월(1월~12월)에 데이터가 존재하는 도매시장만 필터링
sufficient_data_markets = monthly_data_availability[monthly_data_availability.min(axis=1) > 0].index

# 충분한 데이터를 가진 도매시장만 포함
filtered_cabbage_df = cabbage_df[cabbage_df['도매시장'].isin(sufficient_data_markets)]

# 신뢰구간으로 이상치 제거

In [5]:
# 80% 신뢰구간 (하위 10%, 상위 90%) 경계 계산
lower_bound = filtered_cabbage_df['1kg_평균가격'].quantile(0.10)
upper_bound = filtered_cabbage_df['1kg_평균가격'].quantile(0.90)

# 80% 신뢰구간 내의 데이터만 필터링
confidence_filtered_data_80 = filtered_cabbage_df[
    (filtered_cabbage_df['1kg_평균가격'] >= lower_bound) & 
    (filtered_cabbage_df['1kg_평균가격'] <= upper_bound)
]

# 90% 신뢰구간 (하위 5%, 상위 95%) 경계 계산
lower_bound_90 = filtered_cabbage_df['1kg_평균가격'].quantile(0.05)
upper_bound_90 = filtered_cabbage_df['1kg_평균가격'].quantile(0.95)

# 90% 신뢰구간 내의 데이터만 필터링
confidence_filtered_data_90 = filtered_cabbage_df[
    (filtered_cabbage_df['1kg_평균가격'] >= lower_bound_90) & 
    (filtered_cabbage_df['1kg_평균가격'] <= upper_bound_90)
]



In [6]:
# filtered_cabbage_df.to_csv("data-files/filtered_cabbage_df.csv", index=False)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler

# 재배 유형 정의
cultivation_periods = {
    "봄재배": {"planting": [3, 4, 5, 6], "harvesting": [6, 7]},
    "가을재배": {"planting": [8, 9, 10], "harvesting": [10, 11]},
    "월동재배": {"planting": [10, 11], "harvesting": [3, 4, 5]},
    "여름재배": {"planting": [6, 7, 8], "harvesting": [9, 10]}
}

# 각 재배 유형에 대해 파종 및 수확 여부를 나타내는 열 추가
for cultivation, periods in cultivation_periods.items():
    confidence_filtered_data_80[cultivation] = (
        confidence_filtered_data_80['Month'].isin(periods['planting'] + periods['harvesting'])
    ).astype(int)

# '도매시장' 열 원-핫 인코딩 수행
filtered_data_80 = pd.get_dummies(confidence_filtered_data_80, columns=['도매시장'], prefix='Market')

# 피처(feature)와 타겟(target) 선택
features = [
    '1kg_물량', '평균기온', '평균풍속', '평균강수량'
] + list(cultivation_periods.keys()) + [col for col in filtered_data_80.columns if col.startswith('Market_')]
X = filtered_data_80[features]
y = filtered_data_80['1kg_평균가격']

# 결측값 처리: 결측값이 포함된 행 제거
X = X.dropna()
y = y.loc[X.index]

# 피처 스케일링: 데이터 정규화를 위해 StandardScaler 사용
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 데이터를 학습용(80%)과 테스트용(20%)으로 분리
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)



# 랜덤 포레스트

In [31]:
# 랜덤 포레스트 모델 초기화 및 학습
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)

# 테스트 데이터로 예측 수행
y_pred = rf_model.predict(X_test)

# 평가 지표 계산
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# 평가 결과 출력
mae, mse, rmse, r2

(np.float64(190.17654656717903),
 np.float64(90410.36107274547),
 np.float64(300.6831572814571),
 0.43935490243867403)

In [9]:
target_mean = y.mean()

mae_ratio = mae / target_mean

target_mean, mae_ratio

(np.float64(639.6492956891403), np.float64(0.20297687149090024))

In [10]:
from sklearn.model_selection import GridSearchCV

# 랜덤 포레스트 하이퍼파라미터 설정
param_grid = {
    'n_estimators': [50, 100, 200],  # 트리 개수
    'max_depth': [None, 10, 20, 30],  # 최대 깊이
    'min_samples_split': [2, 5, 10],  # 노드 분할을 위한 최소 샘플 수
    'min_samples_leaf': [1, 2, 4]  # 리프 노드의 최소 샘플 수
}

# 랜덤 포레스트 모델 초기화
rf_model = RandomForestRegressor(random_state=42)

# GridSearchCV 설정
grid_search = GridSearchCV(
    estimator=rf_model,
    param_grid=param_grid,
    scoring='neg_mean_absolute_error',  # MAE를 최소화
    cv=3,  # 3-폴드 교차검증
    verbose=2,
    n_jobs=-1  # 병렬 처리
)

# 하이퍼파라미터 튜닝 실행
grid_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터 및 성능 확인
best_params = grid_search.best_params_
best_score = -grid_search.best_score_  # MAE는 음수로 계산되므로 부호 변환

best_params, best_score


Fitting 3 folds for each of 108 candidates, totalling 324 fits


({'max_depth': 20,
  'min_samples_leaf': 1,
  'min_samples_split': 10,
  'n_estimators': 200},
 np.float64(134.99715275448781))

# XGBoost

In [11]:
from xgboost import XGBRegressor

# 랜덤 포레스트 모델 초기화 및 학습
xgb_model = XGBRegressor(random_state=42, objective='reg:squarederror')
xgb_model.fit(X_train, y_train)

# 테스트 데이터로 예측 수행
y_pred = xgb_model.predict(X_test)

# 평가 지표 계산
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# 평가 결과 출력
mae, mse, rmse, r2

(np.float64(130.9840330954913),
 np.float64(28201.557043923094),
 np.float64(167.93319220429026),
 0.4014227127726965)

# 로그

In [12]:
# 1차: 월별, 도매시장별, 산지-광역시도별로 IQR을 이용한 이상치 제거
def remove_outliers_iqr(group, column):
    Q1 = group[column].quantile(0.25)
    Q3 = group[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return group[(group[column] >= lower_bound) & (group[column] <= upper_bound)]

# 그룹화하여 1차 이상치 제거
filtered_cabbage_df_no_outliers_1 = filtered_cabbage_df.groupby(['Month', '도매시장', '산지-광역시도'], group_keys=False).apply(remove_outliers_iqr, column='1kg_평균가격')

# 2차: 로그 변환 적용 후 이상치 제거
filtered_cabbage_df_no_outliers_1['log_1kg_평균가격'] = np.log1p(filtered_cabbage_df_no_outliers_1['1kg_평균가격'])  # 로그 변환

filtered_cabbage_df_log = filtered_cabbage_df_no_outliers_1.groupby(['Month', '도매시장', '산지-광역시도'], group_keys=False).apply(remove_outliers_iqr, column='log_1kg_평균가격')



C:\Users\human\AppData\Local\Temp\ipykernel_832\3434969813.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filtered_cabbage_df_no_outliers_1 = filtered_cabbage_df.groupby(['Month', '도매시장', '산지-광역시도'], group_keys=False).apply(remove_outliers_iqr, column='1kg_평균가격')
C:\Users\human\AppData\Local\Temp\ipykernel_832\3434969813.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filtered_cabbage_df_log = fi

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# 재배 유형 정의
cultivation_periods = {
    "봄재배": {"planting": [3, 4, 5, 6], "harvesting": [6, 7]},
    "가을재배": {"planting": [9, 10], "harvesting": [10, 11]},
    "월동재배": {"planting": [9, 10, 11], "harvesting": [3, 4, 5]},
    "여름재배": {"planting": [6, 7, 8], "harvesting": [9, 10]}
}

# 각 재배 유형에 대해 파종 및 수확 여부를 나타내는 열 추가
for cultivation, periods in cultivation_periods.items():
    filtered_cabbage_df_log[cultivation] = (
        filtered_cabbage_df_log['Month'].isin(periods['planting'] + periods['harvesting'])
    ).astype(int)

# '도매시장' 열 원-핫 인코딩 수행
filtered_data_80 = pd.get_dummies(filtered_cabbage_df_log, columns=['도매시장'], prefix='Market')

# 피처(feature)와 타겟(target) 선택
features = [
    '1kg_물량', '평균기온', '평균풍속', '평균강수량'
] + list(cultivation_periods.keys()) + [col for col in filtered_data_80.columns if col.startswith('Market_')]
X = filtered_data_80[features]
y = filtered_data_80['1kg_평균가격']

# 결측값 처리: 결측값이 포함된 행 제거
X = X.dropna()
y = y.loc[X.index]

# 피처 스케일링: 데이터 정규화를 위해 StandardScaler 사용
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 데이터를 학습용(80%)과 테스트용(20%)으로 분리
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [14]:
# 랜덤 포레스트 모델 초기화 및 학습
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)

# 테스트 데이터로 예측 수행
y_pred = rf_model.predict(X_test)

# 평가 지표 계산
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# 평가 결과 출력
mae, mse, rmse, r2

(np.float64(170.97903660002737),
 np.float64(65113.221925301885),
 np.float64(255.17292553345445),
 0.5232280694588152)

# 1 진행중

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [19]:
data = pd.read_csv("data-files/filtered_90_percent_confidence.csv")

# get_dummies

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler

data['Year'] = pd.to_datetime(data['DATE']).dt.year

cultivation_seasons = {
    '봄재배': {'파종기': [3, 4], '정식기': [4, 5], '수확기': [7, 8]},
    '가을재배': {'파종기': [7], '정식기': [8], '수확기': [10, 11]},
    '월동재배': {'파종기': [9], '정식기': [10], '수확기': [3, 4, 5]},
    '여름재배': {'파종기': [5, 6], '정식기': [6, 7], '수확기': [9]}
}

data['Year'] = pd.to_datetime(data['DATE']).dt.year

# 결과 저장을 위한 데이터프레임 초기화
model_data = pd.DataFrame()

# 재배 방식 정보를 1 또는 0으로 표시
for season, periods in cultivation_seasons.items():
    # 입력 데이터 생성 (파종기 및 정식기)
    input_data = data[data['Month'].isin(periods['파종기'] + periods['정식기'])].copy()
    input_data[f'{season}_여부'] = 1  # 해당 재배 방식 여부를 1로 표시
    
    # 나머지 데이터에 대해서는 0으로 설정
    input_data = data.assign(**{f'{season}_여부': data['Month'].apply(
        lambda x: 1 if x in (periods['파종기'] + periods['정식기']) else 0)})

    # 타겟 데이터 생성 (수확기)
    target_data = data[data['Month'].isin(periods['수확기'])]
    target_values = target_data.groupby(['Year', '도매시장', '산지-광역시도'])['1kg_평균가격'].mean().reset_index()
    target_values.rename(columns={'1kg_평균가격': '타겟값'}, inplace=True)
    
    # 입력 데이터와 타겟 데이터 병합
    season_data = pd.merge(input_data, target_values, on=['Year', '도매시장', '산지-광역시도'], how='inner')
    model_data = pd.concat([model_data, season_data], ignore_index=True)

# 입력 변수(X)와 타겟값(y) 분리
X = model_data[['평균기온', '평균풍속', '평균강수량', '봄재배_여부', '여름재배_여부', '가을재배_여부', '월동재배_여부','도매시장','산지-광역시도']]
y = model_data['타겟값']

# 범주형 데이터 (도매시장, 산지-광역시도)에 대해 원-핫 인코딩
X = pd.get_dummies(X, columns=['도매시장', '산지-광역시도'], drop_first=True)

# 모델 생성 및 성능 평가
# 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 생성 및 학습
model = RandomForestRegressor(random_state=42, n_estimators=100)
model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)

# 평가
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

# 결과 저장 및 표시
performance_metrics = {
    "MAE": mae,
    "RMSE": rmse,
    "R-squared": r2
}

performance_metrics

c:\Users\human\anaconda3\envs\human-da-env\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'MAE': np.float64(45.31967558998742),
 'RMSE': np.float64(86.44003903983291),
 'R-squared': 0.825191363641721}

# 원-핫

In [15]:
# 데이터에 'Year' 열 추가
data['Year'] = pd.to_datetime(data['DATE']).dt.year

# 결과 저장을 위한 데이터프레임 초기화
model_data = pd.DataFrame()

cultivation_seasons = {
    '봄재배': {'파종기': [3, 4], '정식기': [4, 5], '수확기': [7, 8]},
    '가을재배': {'파종기': [7], '정식기': [8], '수확기': [10, 11]},
    '월동재배': {'파종기': [9], '정식기': [10], '수확기': [3, 4, 5]},
    '여름재배': {'파종기': [5, 6], '정식기': [6, 7], '수확기': [9]}
}

# 재배 방식 정보를 추가
for season, periods in cultivation_seasons.items():
    # 입력 데이터 생성 (파종기 및 정식기)
    input_data = data[data['Month'].isin(periods['파종기'] + periods['정식기'])].copy()
    input_data['재배방식'] = season  # 해당 재배 방식 추가
    
    # 타겟 데이터 생성 (수확기)
    target_data = data[data['Month'].isin(periods['수확기'])]
    target_values = target_data.groupby(['Year', '도매시장', '산지-광역시도'])['1kg_평균가격'].mean().reset_index()
    target_values.rename(columns={'1kg_평균가격': '타겟값'}, inplace=True)
    
    # 입력 데이터와 타겟 데이터 병합
    season_data = pd.merge(input_data, target_values, on=['Year', '도매시장', '산지-광역시도'], how='inner')
    model_data = pd.concat([model_data, season_data], ignore_index=True)

# 입력 변수(X)와 타겟값(y) 분리
X = model_data[['평균기온', '평균풍속', '평균강수량', '재배방식', '도매시장', '산지-광역시도']]
y = model_data['타겟값']

# 재배 방식에 대해 원-핫 인코딩 (3개의 열 생성)
X = pd.get_dummies(X, columns=['재배방식', '도매시장', '산지-광역시도'], drop_first=True)

# 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 생성 및 학습
model = RandomForestRegressor(random_state=42, n_estimators=100)
model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)

# 평가
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

# 결과 저장 및 표시
performance_metrics = {
    "MAE": mae,
    "RMSE": rmse,
    "R-squared": r2
}

performance_metrics

c:\Users\human\anaconda3\envs\human-da-env\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'MAE': np.float64(26.394297413081553),
 'RMSE': np.float64(68.45744360078551),
 'R-squared': 0.8994719181902174}